### Assuming undirected, daytime trains including <6> and <7> but not &lt;F&gt;, no skip-stop or peak direction only routes.

In [232]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json
import re
import warnings
from IPython.display import clear_output
import string
from pandas import DataFrame
import types
from itertools import combinations
from typing import Any, Tuple
import typing_extensions

In [233]:
import itertools
def iterator(L):
	a, b = itertools.tee(L)
	next(b, None)
	return list(zip(a,b))

In [234]:
f = open("subwaygraph8to8.json",'r')
G = nx.node_link_graph(json.load(f), directed=False, multigraph=True, edges='edges')

In [235]:
stations = pd.read_csv("MTA_Subway_Stations.csv")

stations = stations[['GTFS Stop ID','Complex ID','Stop Name','Daytime Routes']]
stations.set_index('GTFS Stop ID',inplace=True)
stations.rename_axis("id", axis=0, inplace=True)
stations = stations[stations['Daytime Routes'] != 'SIR']
stations.rename({"Daytime Routes":"lines", "Stop Name": "name"}, axis='columns', inplace=True)
stations.sort_index(inplace=True)
stations['lines'] = [l.split() for l in stations['lines']]


In [236]:
class Pathfinder:
	def __init__(this, start_line):
		this.start_line = start_line
     
	def dijkstra(this, G: nx.Graph, start, end):
		current_line = this.start_line
		prev_line = current_line
		nodes = list(G.nodes)
		inf = float('inf')
		dist_from_start = {n: inf for n in nodes}
		dist_from_start[start] = 0
		predecessors = {n: None for n in nodes}
		best_line = {n: None for n in nodes}

		while len(nodes) > 0:
			candidates = {n: dist_from_start[n] for n in nodes}
			closest = min(candidates, key = candidates.get)  # type: ignore
			for n in G.neighbors(closest):
				wait_time = 0
				try: 
					dist_to_n = G.edges[n, closest, 0]['travel_time'][prev_line]
					d = dist_from_start[closest] + dist_to_n
					if dist_from_start[n] > d:
						dist_from_start[n] = d
						predecessors[n] = closest
						best_line[n] = current_line
						continue
				except KeyError:
					transfers = set(G.nodes[closest]['lines']).intersection(set(G.nodes[n]['lines']))
					newlines = {k: G.nodes[closest]['wait_time'][k] for k in transfers}
					
					try:
						new_line = min(newlines, key = newlines.get)
						wait_time = newlines[new_line]
						prev_line = current_line
						current_line = new_line
					except ValueError:
						next_trains = G.nodes[n]['wait_time']
						new_line = min(next_trains, key=next_trains.get)
						prev_line = current_line
						current_line = new_line
						wait_time = next_trains[current_line]
						
					finally: 
						d = dist_from_start[closest] + dist_to_n + wait_time
						if dist_from_start[n] > d:
							dist_from_start[n] = d
							predecessors[n] = closest
							best_line[n] = current_line
							
			nodes.remove(closest)
		return (dist_from_start, predecessors, best_line)


	def shortestPath(this, G: nx.Graph, start, end):

		distances, predecessors, lines = this.dijkstra(G, start, end)

		if predecessors[end] is None and start != end:
			return [], distances[end]

		path = [end]
		best_line = [lines[end]]
		while path[-1] != start:
			path.append(predecessors[path[-1]])
			best_line.append(lines[path[-1]])
		path.reverse()
		best_line.reverse()

		return path, distances[end], best_line



In [237]:
class Subway:

    def __init__(this, G, station_names: pd.DataFrame):
        """Add a new subway system"""
        this.G = G
        this.stations = station_names

        ## route variables
        this.routes = {}
        this.route_count = 0
    
        

    def find_stop(this, name, line_spec=None):
        '''Helper function to find all stops from a substring of the station name'''
        stations: DataFrame = this.stations
        opts = stations[stations.name.str.contains(name, case=False)]
        if len(opts) == 0: 
            raise RuntimeError("No stations found")
        elif len(opts) == 1:
            return list(opts.index)[0]
        elif len(opts)>1:
            if line_spec:
                line_spec = str(line_spec)
                stopmatch = opts[opts.lines.map(lambda x: line_spec.capitalize() in x)]
                return list(stopmatch.index)[0]
            else:
                print("Multiple stations with name {} found. Select subway line from below".format(name))
                print(opts.head())
                line = str(input("line"))
                if line == 'default': start_id = opts[1]
                else:
                    start_id = opts[opts.lines.map(lambda x: line.capitalize() in x)]
                    if len(start_id) == 0: 
                        warnings.warn("Line not found, defaulting to first station")
                        return list(opts.index)[0]
                    return list(start_id.index)[0]
        else: raise RuntimeError("how did you manage to mess this up either it exists or it doesnt")
                
    def new_route(this, start, stop, start_line=None, stop_line=None, route_name = None, id=False):
        """Adds a route to the route dictionary.

        `start_line` and `stop_line` parameters specify start/stop lines for names that are shared between multiple stops.\n
        `id` parameter takes 3 character GTFS stop IDs, which are different from the MTA Station IDs.
        """
        if not id:
            start = this.find_stop(start, line_spec=start_line)
            stop = this.find_stop(stop, line_spec=stop_line)
            clear_output()
            
        this.route_count += 1
        if route_name: rname = route_name 
        else: rname = "route{}".format(this.route_count)
        this.routes[rname] = (start, stop)

    def new_routes(this, lst, id=False):
        """Bulk add routes from 2 tuples of either stop names or GTFS IDs"""
        for i,j in lst:
            this.new_route(i,j, id=id)

    def remove_route(this, route_name: str):
        '''Remove route by name'''
        try: del this.routes[route_name]
        except KeyError: pass
        
    def remove_routes(this, route_names: list):
        '''Remove routes from list of names'''
        for k in route_names:
            this.remove_route(k)

    def clear_routes(this):
        '''Remove all routes'''
        this.routes = {}
        this.route_count = 0

    
    ## Pathfinding


    def transfer_path(this, route):
        """Modified version of dijkstra path"""
        source = this.routes[route][0]
        target = this.routes[route][1]
        path = Pathfinder(G.nodes[source]['lines'][0])
        path, time, lines = path.shortestPath(G, source, target)
        c = set(combinations(path, 2))
        paths = c - set(iterator(path))  
        for e in paths:
            try: 
                print(e, G.edges[e[0], e[1], 0])
            except KeyError: continue
        pathnames = [[[this.stations.loc[i]['name'],this.stations.loc[j]['name']], list(this.G.edges[i,j,0]['travel_time'].keys())] for i,j in iterator(path)]
        mins = np.ceil(np.ceil(time)/60) # type: ignore
        return mins, pathnames, lines


In [238]:
rng = np.random.default_rng(5368)
stop_pairs_id = rng.choice(list(G.nodes()),size=(3,2), replace=False).tolist()
stop_pairs_name = [[stations.loc[i]['name'] for i in k] for k in stop_pairs_id]
stop_pairs_id
stop_rev = [k[::-1] for k in stop_pairs_id]
stop_rev

[['614', 'F03'], ['109', 'G13'], ['J22', '726']]

In [239]:
nyc = Subway(G, stations)
nyc.new_routes(stop_rev, id=True)
nyc.routes

{'route1': ('614', 'F03'), 'route2': ('109', 'G13'), 'route3': ('J22', '726')}

In [240]:
for k in nyc.routes:
	time, rte, lines = nyc.transfer_path(k)
	break
lines
## don't transfer to the E, figure out how to work around this. 
# should go from lex 63 to lex 59 according to the MTA

# 3rd one is a bit wonky but mta predicts 57 with small delay, likely due to the extra transfer from B/D to M

('626', '629') {'travel_time': {'5': 232.0, '4': 233.0}}
('F06', 'F05') {'travel_time': {'E': 120.0, 'F': 120.0}}


[None,
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 'N',
 'R',
 'E',
 'E',
 'E',
 'E',
 'E',
 'F',
 'F']